## Image-Based Hate Speech Detection Using Autoencoders

### Authors

- Sai Shishir Ailneni
- Priyaanka Reddy Boothkuri
- Manogna Tummanepally

## Introduction

In this notebook, we will develop an image-based model to detect hate speech using autoencoders combined with a classification neural network. This method aims to capture and analyze complex visual patterns that may indicate hate speech in images.

## Import Required Libraries¶
Here, we import all necessary libraries that will be used throughout the notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Input
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score,roc_auc_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

np.random.seed(9012023)

## Dataset Preparation

We'll start by loading our dataset and preparing it through undersampling to ensure each class is equally represented. This helps prevent our model from being biased towards the majority class.


In [2]:
data = pd.read_json('./data/train.jsonl', lines=True)

In [3]:
data.head()

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."


In [4]:
df = pd.DataFrame(data)

# Count the number of samples for each label
label_counts = df['label'].value_counts()

# Find the minimum count of samples among the labels
min_count = label_counts.min()

# Undersample each label to the minimum count
undersampled_dfs = []
for label in df['label'].unique():
    label_df = df[df['label'] == label]
    undersampled_dfs.append(label_df.sample(min_count, random_state=42))

# Concatenate the undersampled dataframes
undersampled_df = pd.concat(undersampled_dfs)

# Shuffle the dataframe to randomize the order of samples
undersampled_df = undersampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the resulting dataframe
undersampled_df['label'].value_counts()

0    3050
1    3050
Name: label, dtype: int64

In [5]:
data['label'].value_counts()

0    5450
1    3050
Name: label, dtype: int64

In [6]:
df = undersampled_df[['label', 'img']]

In [7]:
df.head()

,label,img
0,0,img/36780.png
1,0,img/76098.png
2,1,img/21408.png
3,0,img/98071.png
4,1,img/68120.png


In [8]:
df['img'] = './data/' + df['img']

/var/folders/jj/rrmlwr416gd3q6qsbg5br2ww0000gn/T/ipykernel_50921/889627064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['img'] = './data/' + df['img']


In [9]:
# Split the dataframe df into train and test using train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Extract image paths and labels from the split DataFrames
train_image_paths = train_df['img'].tolist()
train_labels = train_df['label'].tolist()

test_image_paths = test_df['img'].tolist()
test_labels = test_df['label'].tolist()

## Image Preprocessing

Next, we define a function to load and preprocess images. This includes resizing and normalizing the images to ensure they are in a format suitable for neural network processing.

In [10]:
# Define a function to load and preprocess images and convert them to numpy arrays
def load_and_preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Assuming images are in RGB format
    image = tf.image.resize(image, [224, 224])  # Specify desired height and width
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    return image.numpy(), label

In [11]:
# Load and preprocess images for training and testing sets and convert them to numpy arrays
x_train, y_train = [], []
for image_path, label in zip(train_image_paths, train_labels):
    image, label = load_and_preprocess_image(image_path, label)
    x_train.append(image)
    y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test, y_test = [], []
for image_path, label in zip(test_image_paths, test_labels):
    image, label = load_and_preprocess_image(image_path, label)
    x_test.append(image)
    y_test.append(label)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [12]:
print(x_train.shape)

(4880, 224, 224, 3)


In [13]:
print(x_test.shape)
print(y_test.shape)

(1220, 224, 224, 3)
(1220,)


In [14]:
x_train_gray = np.mean(x_train, axis=-1)
x_test_gray = np.mean(x_test, axis=-1)

In [15]:
print(x_train_gray.shape)
print(x_test_gray.shape)

(4880, 224, 224)
(1220, 224, 224)


In [16]:
x_train_reshaped = np.reshape(x_train_gray, (-1, 224*224))
x_test_reshaped = np.reshape(x_test_gray, (-1, 224*224))

## Model Building: Autoencoder

We are using an autoencoder for dimensionality reduction, which helps in extracting meaningful features from images that are useful for classification.

In [17]:
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Input, Dense

input_dim = 224 * 224
latent_vec_len = 32  # this is the 'bottleneck' of the autoencoder

# Define the autoencoder architecture
# build the encoder
autoencodernew = Sequential()
autoencodernew.add(Input(shape=(input_dim,)))
autoencodernew.add(Dense(256, activation='relu'))
autoencodernew.add(Dense(128, activation='relu'))
autoencodernew.add(Dense(64, activation='relu'))

# the bottleneck layer
autoencodernew.add(Dense(latent_vec_len, activation='relu'))

# the decoder layer
autoencodernew.add(Dense(64, activation='relu'))
autoencodernew.add(Dense(128, activation='relu'))
autoencodernew.add(Dense(256, activation='relu'))
autoencodernew.add(Dense(input_dim))

# NOTE: The output layer of a Keras autoencoder includes a sigmoid activation function
# because it forces the output to be in the range [0, 1]. This is done to ensure that the 
# output can be interpreted as a probability or a pixel intensity of a grayscale image.

optimizer = Adam()

# Compile the autoencoder model
autoencodernew.compile(loss='binary_crossentropy', optimizer=optimizer)

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Get summary
autoencodernew.summary()

# Now, when you train your model, you can pass `early_stopping` as a callback
# For example:
# autoencodernew.fit(x_train, x_train, validation_data=(x_val, x_val), epochs=100, callbacks=[early_stopping])


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │    12,845,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 50176)          │    12,895,232 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,827,232 (98.52 MB)

 Trainable params: 25,827,232 (98.52 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
autoencodernew.fit(x_train_reshaped, x_train_reshaped, epochs=50, batch_size=32, validation_data=(x_test_reshaped, x_test_reshaped))

Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - loss: 1.7813 - val_loss: 0.8329
Epoch 2/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 12s 81ms/step - loss: 0.7652 - val_loss: 0.6856
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - loss: 0.6882 - val_loss: 0.6638
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 12s 80ms/step - loss: 0.6633 - val_loss: 0.6436
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - loss: 0.6392 - val_loss: 0.6295
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 13s 82ms/step - loss: 0.6613 - val_loss: 0.6996
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 12s 80ms/step - loss: 0.6720 - val_loss: 0.6335
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 12s 81ms/step - loss: 0.6291 - val_loss: 0.6231
Epoch 9/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 12s 81ms/step - loss: 0.6325 - val_loss: 0.6195
Epoch 10/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - loss: 0.6211 - val_loss: 0.6180
Epoch 11/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 12s 77ms/step - loss: 0.6163 - val_loss: 0.6147
Epoch 12/50
153/153 ━━━━━━━━━━

In [19]:
encoder = Sequential(autoencodernew.layers[:4])  # Get encoder part of autoencoder

## Classification Model

After training the autoencoder, we use the encoded features to train a classifier to predict the presence of hate speech in images.

In [20]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Freeze encoder layers
for layer in encoder.layers:
    layer.trainable = False

# Create the model
model = tf.keras.Sequential([
    encoder,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Get summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,888,544 (49.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 12,888,544 (49.17 MB)

In [21]:
model.fit(x_train_reshaped, y_train, epochs=50, batch_size=32, validation_data=(x_test_reshaped, y_test))

Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4936 - loss: 0.7236 - val_accuracy: 0.4943 - val_loss: 0.6944
Epoch 2/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5066 - loss: 0.6973 - val_accuracy: 0.4975 - val_loss: 0.6940
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4974 - loss: 0.6937 - val_accuracy: 0.5221 - val_loss: 0.6935
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5099 - loss: 0.6939 - val_accuracy: 0.5197 - val_loss: 0.6920
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5163 - loss: 0.6932 - val_accuracy: 0.5131 - val_loss: 0.6926
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5018 - loss: 0.6937 - val_accuracy: 0.5107 - val_loss: 0.6943
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4940 - loss: 0.6939 - val_accuracy: 0.5205 - val_loss: 0.6929
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5108 - loss: 0.6927 - val_accuracy: 0

## Model Evaluation

Finally, we evaluate our classifier on the test set and display key metrics such as accuracy, precision, recall, and F1-score.

In [22]:
# Get predictions for the test data
y_pred = model.predict(x_test_reshaped)
y_pred_classes = (y_pred > 0.5).astype("int32")  # Convert probabilities to binary classes

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
conf_matrix

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[558,  66],
       [530,  66]])

In [23]:
accuracy = accuracy_score(y_test,y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)
AUC = roc_auc_score(y_test,y_pred_classes)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Area Under Curve:", AUC)

Accuracy: 0.5114754098360655
Precision: 0.5
Recall: 0.11073825503355705
F1 Score: 0.18131868131868134
Area Under Curve: 0.5024845121321632
